In [71]:
import mm_predictor # stat-analysis library
from sklearn import cross_validation, linear_model
import csv
import random
import numpy as np
import pandas as pd
#from xgboost import XGBClassifier

In [72]:
# intialize stat & elo dictionaries
mm_predictor.init()

In [73]:
# Load data
season_data = pd.read_csv('./ncaa-data/RegularSeasonDetailedResults.csv')
tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
tourney_data = tourney_data[tourney_data.Season != 2017]
season_data = season_data[season_data.Season > 2010]

In [74]:
aggregated_data = pd.concat([season_data, tourney_data])

X,Y = mm_predictor.analyze_teams_diff(aggregated_data)
# Run script

/Users/shaheensharifian/Documents/Info370/info370-FinalProject/mm_predictor.py:83: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  winAd['fgp'] = winAd['WFGM'] / winAd['WFGA'] * 100
/Users/shaheensharifian/Documents/Info370/info370-FinalProject/mm_predictor.py:84: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  winAd['3pp'] = winAd['WFGM3'] / winAd['WFGA3'] * 100
/Users/shaheensharifian/Documents/Info370/info370-FinalProject/mm_predictor.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using

In [75]:
from sklearn.linear_model import LogisticRegression
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import GaussianNB
from sklearn.pipeline import make_pipeline, make_union
from sklearn.preprocessing import RobustScaler
from sklearn.svm import LinearSVC
from tpot.builtins import StackingEstimator
from xgboost import XGBClassifier

# NOTE: Make sure that the class is labeled 'target' in the data file
# tpot_data = pd.read_csv('PATH/TO/DATA/FILE', sep='COLUMN_SEPARATOR', dtype=np.float64)
# features = tpot_data.drop('target', axis=1).values
# training_features, testing_features, training_target, testing_target = \
#             train_test_split(features, tpot_data['target'].values, random_state=42)

# Score on the training set was:-0.5443811951009836
exported_pipeline = make_pipeline(
    StackingEstimator(estimator=XGBClassifier(learning_rate=0.01, max_depth=2, min_child_weight=4, n_estimators=100, nthread=1, subsample=0.15000000000000002)),
    RobustScaler(),
    StackingEstimator(estimator=GaussianNB()),
    StackingEstimator(estimator=LinearSVC(C=15.0, dual=True, loss="hinge", penalty="l2", tol=0.0001)),
    StackingEstimator(estimator=GaussianNB()),
    LogisticRegression(C=5.0, dual=False, penalty="l1")
)

# exported_pipeline = LogisticRegression()

exported_pipeline.fit(np.array(X), np.array(Y))


LogisticRegression(C=1.0, class_weight=None, dual=False, fit_intercept=True,
          intercept_scaling=1, max_iter=100, multi_class='ovr', n_jobs=1,
          penalty='l2', random_state=None, solver='liblinear', tol=0.0001,
          verbose=0, warm_start=False)

In [76]:
# Create id to team name mappings
teamNames = pd.read_csv('./ncaa-data/Teams.csv')
def teamIDNameMapping():
    team_id_map = {}
    for index, row in teamNames.iterrows():
        team_id_map[row['TeamID']] = row['TeamName']
    
    return team_id_map

team_id_map = teamIDNameMapping()

In [77]:
def WinChanceMapping(year):
    
    submission_data = []
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    tourney_teams = tourney_seeds[tourney_seeds['Season'] == year]['Team']

    stat_fields = ['score', 'OR', 'DR', 'Ast', 'TO', 'Stl', 'Blk', 'PF',
                   'fgp', '3pp', 'ftp']
    
    tourney_teams.sort_values()
    
    for team_1 in tourney_teams:
        for team_2 in tourney_teams:
            if team_1 != team_2:
                prediction = mm_predictor.predict_winner(
                    team_1, team_2, exported_pipeline, 2017, stat_fields)
                label = '2017_' + str(team_1) + '_' + str(team_2)
                print(prediction[0][0])
                submission_data.append([label, prediction[0][0]])
    
    
    teamTeamWinChanceMap = {}
    for line in submission_data:
        title = line[0].split('_')
        team1 = team_id_map[int(title[1])]
        team2 = team_id_map[int(title[2])]
        winRate = str(round(line[1]* 100,2))
        if team1 not in teamTeamWinChanceMap:
            teamTeamWinChanceMap[team1] = {}
        if float(winRate) >= 50:
            if team1 in teamTeamWinChanceMap and team2 in teamTeamWinChanceMap[team1] and teamTeamWinChanceMap[team1][team2] > float(winRate):
                continue
            if team2 in teamTeamWinChanceMap and team1 in teamTeamWinChanceMap[team2] and teamTeamWinChanceMap[team2][team1] < float(winRate):
                # update teamTeammap[team2][team1] to remove that key
                teamTeamWinChanceMap[team2].pop(team1, None)
            teamTeamWinChanceMap[team1][team2] = float(winRate)
            
    return teamTeamWinChanceMap

In [78]:
# print("Writing %d results." % len(submission_data))
# with open('./ncaa-data/submission.csv', 'w') as f:
#     writer = csv.writer(f)
#     writer.writerow(['id', 'pred'])
#     writer.writerows(submission_data)

chanceMap = WinChanceMapping(2017)

7.1054273576e-14
1.0
0.999999999972
1.0
0.999999999996
1.0
1.0
0.0328064018384
2.22044604925e-16
1.0
0.00726076943996
0.0229663681588
0.0339728155298
1.0
1.0
1.0
1.0
1.30118138486e-13
0.999834663529
0.0
0.0
4.97379915032e-14
1.0
0.999999999998
0.999999999764
0.99999999999
0.999999999031
0.735053075621
1.0
0.999999895485
0.999999999232
0.999999999711
1.0
5.77315972805e-15
0.99999997164
1.0
1.83528288933e-05
8.25983725861e-12
0.0
1.0
1.0
1.0
0.0
0.576997840201
0.996087032471
1.0
0.999999998923
1.0
1.0
1.0
0.0
0.0
0.0
0.999993728408
0.225485485007
1.0
0.999995096637
1.27959204343e-08
0.0575646429604
2.95298761332e-07
0.999999999997
1.31539223958e-11
0.999999978194
0.999925085262
0.998358011898
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.999999999998
0.00444796613657
1.0
0.99999999999
0.999999999997
0.999999999998
1.0
1.0
1.0
1.0
0.639964829908
1.0
8.60300165895e-06
8.94406326779e-10
0.404893885993
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
1.0
0.0731365895983
1.0
1.0
0.999999996009
0.991209562025


0.987068826283
1.65695140714e-05
0.00542617048354
0.10071980452
0.0
0.0
0.999999999929
0.0
0.0
0.0
0.995832513683
0.999999995101
1.0
0.999999999987
0.0
3.81028542051e-13
0.0
0.0
0.0
2.75390043908e-05
2.66976402097e-07
6.4263582904e-06
6.50508691358e-08
2.22044604925e-16
0.999997596227
6.02968786012e-10
8.20028841497e-08
2.18124358464e-07
0.999999979995
0.0
2.22214113776e-09
0.0104546701012
0.0
0.0
0.0
0.98199972987
0.600474595043
0.998621887767
0.0
0.0
1.59872115546e-14
0.107664904204
5.85299332334e-08
0.999911712371
0.999958340817
0.99999999619
0.0
0.0
0.0
1.00481845067e-11
0.0
0.0194619701295
1.28521637777e-11
0.0
0.0
0.0
1.84217434226e-05
0.0
2.8899269644e-09
8.41104963456e-13
3.8413716652e-14
0.00501919524097
0.98434595239
2.1551649354e-12
0.0
0.978544112423
0.0736540497524
0.999999628238
0.368649157172
0.994825540326
0.9997466945
7.52731210696e-14
0.0
1.0
1.62092561595e-14
5.21804821574e-14
7.81597009336e-14
0.999999881242
1.0
1.0
1.0
0.0
1.34294304566e-08
0.0
0.0
0.0
0.9999707600

1.0
0.0
0.0
2.30224625117e-05
4.01375864767e-08
0.575891027916
2.94761122954e-07
9.70437615198e-05
0.0019884230385
0.0
0.0
0.999999995992
0.0
0.0
0.0
0.809550987383
0.999999724593
1.0
0.999999999295
0.0
6.88338275268e-15
0.0
0.0
0.0
0.0169709218932
4.89906661683e-07
4.7492645372e-09
1.14319690159e-07
1.15719478444e-09
0.0
0.999864891558
1.07263087301e-11
1.45875556079e-09
3.88023146947e-09
0.999998875427
0.0
3.9529934881e-11
0.000187908483041
0.0
0.0
0.0
0.0260402184621
0.928008365143
0.0
0.0
2.22044604925e-16
0.00214174814485
1.04119335376e-09
0.995061050748
0.997663533426
0.999999785847
0.0
0.0
0.0
1.78745906965e-13
0.0
0.000352957493793
2.28705943073e-13
0.0
0.0
0.0
3.27711702108e-07
0.0
5.14090991999e-11
1.487698853e-14
6.66133814775e-16
8.97291964967e-05
0.5279899852
0.0
0.0
0.000834989661741
1.45691008246e-06
0.9801147092
1.06991134361e-05
0.003510464778
0.0674421052224
0.0
0.0
0.99999999989
0.0
0.0
0.0
0.993560579843
0.999999992413
1.0
0.999999999981
0.0
2.46025422257e-13
0.0
0.

1.0
1.0
1.0
1.0
1.0
0.999999890323
1.0
1.0
1.0
1.0
1.0
1.89957755081e-08
1.0
1.0
0.983691132083
2.71444627321e-05
1.99840144433e-15
1.0
1.0
1.0
0.0
0.999999776928
0.999999998805
1.0
1.0
1.0
1.0
1.0
3.1241675913e-13
0.0
0.0
0.999999999998
0.999998954831
1.0
0.999999999999
0.0403557792741
0.999995018391
1.0
4.3227234199e-05
1.0
0.999999999977
0.9999999995
1.0
1.0
3.22186721746e-12
0.0
0.98554490245
0.106235133532
0.999999751319
0.466068350173
0.996532725865
0.999830542976
1.12576614697e-13
0.0
1.0
2.42028619368e-14
7.79376563287e-14
1.16795462191e-13
0.99999992056
1.0
1.0
1.0
0.0
2.00760863489e-08
0.0
0.0
0.0
0.999980440452
0.591969761443
0.0138692970781
0.252919408898
0.00341518610638
9.21041021229e-12
0.999999999954
3.17634753121e-05
0.00430134287018
0.0113602867075
1.0
0.0
0.000117049014791
0.99820651074
0.0
0.0
0.0
0.999999652047
0.999987370176
0.999999973804
0.0
4.52837767284e-12
8.45104874969e-10
0.999842696453
0.00307388775008
0.999999998324
0.999999999209
1.0
0.0
0.0
0.0
5.293449

In [79]:
def filterTourneyDataByYear(year):

    tourney_data = pd.read_csv('./ncaa-data/NCAATourneyDetailedResults.csv')
    tourney_data = tourney_data[tourney_data.Season == year]
    
    return tourney_data

def getWinnersList(tourney_data):
    winners = []

    tourney_data_grouped = tourney_data[4:].groupby('WTeamID').size().reset_index(name='NumWins')
    for index, row in tourney_data_grouped.iterrows():
        teamName = team_id_map[row['WTeamID']]
        wins = row['NumWins']
        if len(winners) == 0:
            winners.append([])
        winners[0].append(teamName)
        if wins > 1:
            if len(winners) == 1:
                winners.append([])
            winners[1].append(teamName)
        if wins > 2:
            if len(winners) == 2:
                winners.append([])
            winners[2].append(teamName)
        if wins > 3:
            if len(winners) == 3:
                winners.append([])
            winners[3].append(teamName)
        if wins > 4:
            if len(winners) == 4:
                winners.append([])
            winners[4].append(teamName)
        if wins > 5:
            if len(winners) == 5:
                winners.append([])
            winners[5].append(teamName)
        if wins > 6:
            if len(winners) == 6:
                winners.append([])
            winners[6].append(teamName)
    
    return winners

In [80]:
# Method that takes in season's tournament data, id to name mapping of teams
def calcBracketScore(teamTeamWinChanceMap, tourney_data):
    
    winners = getWinnersList(tourney_data)
    
    # First four rows is 'first four' and not in actual bracket
    firstFour = tourney_data[:4]
    mainTourney = tourney_data[4:]
    
    tourney_seeds = pd.read_csv('./ncaa-data/TourneySeeds.csv')
    
    tourney = [-1] * max(tourney_seeds['Team'])

    # Look at first four teams
    for index, row in firstFour.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]):
            tourney[index2] = index1
        else:
            tourney[index1] = index2

    # MAIN TOURNAMENT 
    score = 0
    for index, row in mainTourney.iterrows():
        index1 = int(row['WTeamID'])
        index2 = int(row['LTeamID'])
        while tourney[index1] > 0:
            index1 = tourney[index1]
        while tourney[index2] > 0:
            index2 = tourney[index2]
        team1Name = team_id_map[index1]
        team2Name = team_id_map[index2]

        if (team1Name in teamTeamWinChanceMap) and (team2Name in teamTeamWinChanceMap[team1Name]): # team1 would win
            tourney[index2] = index1
            tourney[index1] = tourney[index1] - 1
            if team1Name in winners[abs(tourney[index1]) - 2]:
                score += 2**(abs(tourney[index1]) - 2) * 10
                print(team1Name + ' vs ' + team2Name + ', team 1 wins')
                print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
            else: # Delete else statemenet when done debugging
                print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team1 wins')

        else: #team2 would win
            tourney[index1] = index2
            tourney[index2] = tourney[index2] - 1
            if team2Name in winners[abs(tourney[index2]) - 2]:
                score += 2**(abs(tourney[index2]) - 2) * 10
                print(team1Name + ' vs ' + team2Name + ', team 2 wins')
                print('Score ' + str(2**(abs(tourney[index1]) - 2) * 10))
            else: # Delete else statement when done debugging
                print('Incorrect: Predicted ' + team1Name + ' vs ' + team2Name + ', team2 wins')

    return score

In [81]:
data_2017 = filterTourneyDataByYear(2017)
data_2017
print(calcBracketScore(chanceMap, data_2017))

Arizona vs North Dakota, team 1 wins
Score 10
Incorrect: Predicted Butler vs Winthrop, team2 wins
Incorrect: Predicted Florida vs ETSU, team2 wins
Florida St vs FL Gulf Coast, team 1 wins
Score 10
Gonzaga vs S Dakota St, team 1 wins
Score 10
Iowa St vs Nevada, team 1 wins
Score 10
Incorrect: Predicted MTSU vs Minnesota, team2 wins
Northwestern vs Vanderbilt, team 1 wins
Score 10
Notre Dame vs Princeton, team 1 wins
Score 10
Purdue vs Vermont, team 1 wins
Score 10
Incorrect: Predicted St Mary's CA vs VA Commonwealth, team2 wins
Villanova vs New Orleans, team 1 wins
Score 10
Incorrect: Predicted Virginia vs UNC Wilmington, team2 wins
West Virginia vs Bucknell, team 1 wins
Score 10
Incorrect: Predicted Wisconsin vs Virginia Tech, team2 wins
Xavier vs Maryland, team 1 wins
Score 10
Arkansas vs Seton Hall, team 1 wins
Score 10
Baylor vs New Mexico St, team 1 wins
Score 10
Incorrect: Predicted Cincinnati vs Wake Forest, team2 wins
Duke vs Troy, team 1 wins
Score 10
Kansas vs NC Central, team

In [82]:
from sklearn.metrics import make_scorer
bballScorer = make_scorer(calcBracketScore, greater_is_better=True)

In [83]:
chanceMap

{'Arizona': {'Baylor': 100.0,
  'Bucknell': 99.93,
  'Butler': 96.24,
  'Cincinnati': 100.0,
  'Dayton': 97.04,
  'FL Gulf Coast': 100.0,
  'Florida': 100.0,
  'Iona': 100.0,
  'Jacksonville St': 100.0,
  'Kansas St': 100.0,
  'Louisville': 99.98,
  'MTSU': 99.99,
  'Maryland': 100.0,
  'Miami FL': 100.0,
  'Michigan': 100.0,
  'Michigan St': 100.0,
  "Mt St Mary's": 100.0,
  'N Kentucky': 100.0,
  'NC Central': 100.0,
  'New Mexico St': 100.0,
  'New Orleans': 100.0,
  'North Dakota': 100.0,
  'Northwestern': 100.0,
  'Oregon': 100.0,
  'Princeton': 100.0,
  'Providence': 100.0,
  'S Dakota St': 100.0,
  'SMU': 100.0,
  'South Carolina': 100.0,
  "St Mary's CA": 100.0,
  'TX Southern': 100.0,
  'Troy': 100.0,
  'UC Davis': 100.0,
  'VA Commonwealth': 100.0,
  'Vanderbilt': 100.0,
  'Vermont': 100.0,
  'Virginia': 100.0,
  'Winthrop': 68.17,
  'Wisconsin': 100.0},
 'Arkansas': {'Arizona': 100.0,
  'Baylor': 100.0,
  'Bucknell': 100.0,
  'Butler': 100.0,
  'Cincinnati': 100.0,
  'Dayton